Import Libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

Set the url

In [11]:
url = "https://tracker.gg/valorant/leaderboards/ranked/pc/default?region=na&page=1&act=dcde7346-4085-de4f-c463-2489ed47983b"

Function

In [12]:
# Declare a function
def Top15Players():

  #print("Hi")
  # Initiate empty lists to hold data
  player_links = []
  player_tags = []
  Names = []
  Damage = [] #4
  Kills = []#15
  KDR = [] #5
  WinRate = [] #7
  Wins = [] #8

  # Reach out to server
  page = requests.get(url)

  print(page.status_code)

  # If successfully reached the server then execute code
  if page.status_code == 200:
    print("Successful")

    # Update code to BeautifulSoup
    MainSoup = BeautifulSoup(page.content, "html.parser")

    # Find all td with the class username on the page
    player_cells = MainSoup.find_all("td", class_ = "username", limit = 15)

    # Look for each players unique link extension
    for player_cell in player_cells:
      player_link_tag = player_cell.find("a")
      if player_link_tag:
        player_tag = player_link_tag.get("href")
        player_tags.append(player_tag)

    # Insert each players unique link ID to get the link to their individual profile page
    for tags in player_tags:
      player_link = "https://tracker.gg" + tags + "/overview"
      player_links.append(player_link)

    # Reach out to server for each players profile page
    for player in player_links:
      response = requests.get(player)
      soup = BeautifulSoup(response.content, "html.parser")

      # Search for the player's username and add it to a list of names
      name_span = soup.find("span", class_ = "trn-ign__username")
      if name_span:
        name = name_span.text
        Names.append(name)

      # Search player's profile for their stats
      stats = soup.find_all("span", class_ = "value", limit = 20)
      if stats:
        damage = stats[3].text
        kdr = stats[4].text
        kills = stats[15].text
        winrate = stats[6].text
        win = stats[7].text
        Damage.append(damage)
        Kills.append(kills)
        KDR.append(kdr)
        WinRate.append(winrate)
        Wins.append(win)

      player_stats = {
          "Player" : Names,
          "Avg Damage" : Damage,
          "Avg Kills" : Kills,
          "Kill/ Death" : KDR,
          "Win Rate" : WinRate,
          "Wins" : Wins
      }

  else:
    print('Error')
  players_df = pd.DataFrame(player_stats)

  stats = ["Avg Damage", "Avg Kills", "Kill/ Death", "Wins"]
  summary = players_df[stats].apply(pd.to_numeric, errors="coerce").describe()
  print(summary)

  players_df.to_csv("Player_Stats.csv", index = False)
  #players_df.to_csv("Player_Stats.csv", index = False)

  return players_df

  TopPlayers = Top15Players()

In [13]:
Top15Players()

403
Error


UnboundLocalError: local variable 'player_stats' referenced before assignment